In [1]:
import pandas as pd
from functions import assembleDf, epochG, epochsG
from datetime import datetime, timedelta
import numpy as np
import sys
sys.path.append('..')
from pyglicko2.glicko2_tests import exampleCase
from pyglicko2.glicko2 import Player
import glicko2
import time


In [2]:
matches = pd.read_csv('../Data/matches_glicko2.csv',parse_dates = ['tourney_date'], 
                      infer_datetime_format = True)

Because memory constraints limit the batch size in epochsG, I will need to process incrementally.

In [3]:
# base batches.  can be further broken up by splitBatch
eg_batches_50 = [(s,f) for s,f in zip(range(0, 876_978-50_000,50_000),
                      range(50_000, 876978,50_000))] + [(850_000,876979)]
eg_batches_100 = [(s,f) for s,f in zip(range(0, 876_978-100_000,100_000),
                      range(100_000, 876978,100_000))] + [(800_000,876979)]

ratings_histories = []
players_dict = {}

In [4]:
def ceildiv(a, b):
    return -(a // -b)

In [5]:
def splitBatch(atuple, divisions):
    '''Break up batches into smaller pieces to allow epochsG to run.'''
    batch_size = ceildiv((atuple[1] - atuple[0]),divisions)
    if batch_size < (atuple[1] - atuple[0]):
        batches = [b for b in zip(range(atuple[0],atuple[1]-batch_size,batch_size),
             range(atuple[0]+batch_size,atuple[1],batch_size))]
        extra = [(batches[-1][1],atuple[1])]
        return batches + extra
    else:
        return atuple

In [6]:
%%time
for batch in eg_batches_50[0:1]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 19.6 s, sys: 125 ms, total: 19.8 s
Wall time: 19.7 s


In [7]:
%%time

for batch in eg_batches_50[1:2]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 7.36 s, sys: 30.7 ms, total: 7.39 s
Wall time: 7.38 s


In [8]:
%%time

for batch in eg_batches_50[2:3]:
    players_dict,rh = epochsG(matches[batch[0]:batch[1]],players_dict,365)
    ratings_histories += [rh]

CPU times: user 7.59 s, sys: 33.1 ms, total: 7.62 s
Wall time: 7.61 s


In [9]:
eg_batches_50[3]

(150000, 200000)

In [10]:
b = splitBatch(eg_batches_50[3],2); b0 = b[0]; b1 = b[1]
b

[(150000, 175000), (175000, 200000)]

In [11]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 3.61 s, sys: 11.7 ms, total: 3.63 s
Wall time: 3.62 s


In [12]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 3.56 s, sys: 16.5 ms, total: 3.58 s
Wall time: 3.58 s


In [13]:
splitBatch(eg_batches_50[4],3)

[(200000, 216667), (216667, 233334), (233334, 250000)]

In [14]:
b = splitBatch(eg_batches_50[4],4); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3=b[3]
b

[(200000, 212500), (212500, 225000), (225000, 237500), (237500, 250000)]

In [15]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.96 s, sys: 13.5 ms, total: 1.97 s
Wall time: 1.97 s


In [16]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.83 s, sys: 15.5 ms, total: 1.85 s
Wall time: 1.84 s


In [17]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.91 s, sys: 15.2 ms, total: 1.93 s
Wall time: 1.92 s


In [18]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 2.05 s, sys: 14.9 ms, total: 2.07 s
Wall time: 2.07 s


In [19]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[5][0]:eg_batches_50[5][1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 6.88 s, sys: 40.7 ms, total: 6.92 s
Wall time: 6.91 s


%%time
players_dict,rh = epochsG(matches[eg_batches_50[6][0]:eg_batches_50[6][1]],players_dict,365)
ratings_histories += [rh]

In [20]:
b = splitBatch(eg_batches_50[6],8); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3=b[3];
b4 = b[0]; b5 = b[1]; b6 = b[2]; b7=b[3]
b

[(300000, 306250),
 (306250, 312500),
 (312500, 318750),
 (318750, 325000),
 (325000, 331250),
 (331250, 337500),
 (337500, 343750),
 (343750, 350000)]

In [21]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.46 s, sys: 12.4 ms, total: 1.48 s
Wall time: 1.47 s


In [22]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.56 s, sys: 25.9 ms, total: 1.59 s
Wall time: 1.57 s


In [23]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.33 s, sys: 9.24 ms, total: 1.34 s
Wall time: 1.34 s


In [24]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.48 s, sys: 10.4 ms, total: 1.49 s
Wall time: 1.49 s


In [25]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.46 s, sys: 13 ms, total: 1.47 s
Wall time: 1.47 s


In [26]:
%%time
players_dict,rh = epochsG(matches[b5[0]:b5[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.56 s, sys: 11.5 ms, total: 1.57 s
Wall time: 1.57 s


In [27]:
%%time
players_dict,rh = epochsG(matches[b6[0]:b6[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.38 s, sys: 26.7 ms, total: 1.41 s
Wall time: 1.39 s


In [28]:
%%time
players_dict,rh = epochsG(matches[b7[0]:b7[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.48 s, sys: 17 ms, total: 1.5 s
Wall time: 1.49 s


%%time

players_dict,rh = epochsG(matches[eg_batches_50[7][0]:eg_batches_50[7][1]],players_dict,365)
ratings_histories += [rh]

In [29]:
b = splitBatch(eg_batches_50[7],7); 
b0 = b[0]; b1 = b[1]; b2 = b[2]; b3 = b[3]; b4 = b[4]; b5 = b[5];
b6 = b[6]


In [30]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.61 s, sys: 15.4 ms, total: 1.62 s
Wall time: 1.62 s


In [31]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.51 s, sys: 12.9 ms, total: 1.53 s
Wall time: 1.52 s


In [32]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.68 s, sys: 18.4 ms, total: 1.7 s
Wall time: 1.69 s


In [33]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.68 s, sys: 10.1 ms, total: 1.69 s
Wall time: 1.69 s


In [34]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.72 s, sys: 14.2 ms, total: 1.74 s
Wall time: 1.73 s


In [35]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.52 s, sys: 13.8 ms, total: 1.53 s
Wall time: 1.53 s


In [36]:
%%time
players_dict,rh = epochsG(matches[b5[0]:b5[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.74 s, sys: 16.9 ms, total: 1.75 s
Wall time: 1.74 s


In [37]:
%%time
players_dict,rh = epochsG(matches[b6[0]:b6[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.72 s, sys: 13.8 ms, total: 1.74 s
Wall time: 1.73 s


In [38]:
b = splitBatch(eg_batches_50[8],2); b0 = b[0]; b1 = b[1]
b

[(400000, 425000), (425000, 450000)]

In [39]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 3.72 s, sys: 27.6 ms, total: 3.75 s
Wall time: 3.74 s


In [40]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 3.41 s, sys: 19.7 ms, total: 3.43 s
Wall time: 3.43 s


In [41]:
b = splitBatch(eg_batches_50[9],8); b0 = b[0]; b1 = b[1]; 
b2 = b[2];b3 = b[3];b4 =b[4]; b5 = b[5]; b6 = b[6]; b7 = b[7]
b

[(450000, 456250),
 (456250, 462500),
 (462500, 468750),
 (468750, 475000),
 (475000, 481250),
 (481250, 487500),
 (487500, 493750),
 (493750, 500000)]

In [42]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.53 s, sys: 14.4 ms, total: 1.54 s
Wall time: 1.54 s


In [43]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

CPU times: user 1.7 s, sys: 10.6 ms, total: 1.71 s
Wall time: 1.7 s


In [ ]:
%%time
players_dict,rh = epochsG(matches[b2[0]:b2[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b3[0]:b3[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b4[0]:b4[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b5[0]:b5[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b6[0]:b6[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b7[0]:b7[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
b = splitBatch(eg_batches_50[10],2); b0 = b[0]; b1 = b[1]
b

In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
b = splitBatch(eg_batches_50[11],5); b0 = b[0]; b1 = b[1];
b2 = b[2]; b3 = b[3];b4 = b[4];
b

In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [66]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

ZeroDivisionError: float division by zero

In [67]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[11][0]:eg_batches_50[11][1]],players_dict,365)
ratings_histories += [rh]

ZeroDivisionError: float division by zero

In [ ]:
b = splitBatch(eg_batches_50[12],2); b0 = b[0]; b1 = b[1]
b

In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[13][0]:eg_batches_50[13][1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
b = splitBatch(eg_batches_50[14],2); b0 = b[0]; b1 = b[1]
b

In [ ]:
%%time
players_dict,rh = epochsG(matches[b0[0]:b0[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[b1[0]:b1[1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[15][0]:eg_batches_50[15][1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[16][0]:eg_batches_50[16][1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
%%time
players_dict,rh = epochsG(matches[eg_batches_50[17][0]:eg_batches_50[17][1]],players_dict,365)
ratings_histories += [rh]

In [ ]:
rh